# NBA — Part 1: Setup & First SQL workflows

Goals
- Connect your local dataset to a lightweight SQL workflow (SQLite) and run initial aggregations useful for feature engineering.
- Produce a repeatable notebook that: creates/opens a DB, ingests CSVs (if present), and runs example SQL queries to build season-level summaries.

In [3]:
# Imports and display settings
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)
repo_root = Path.cwd()
repo_root

PosixPath('/Users/darshj/NBA-Contract-Predictor/Baseball/NBA-Contract-Predictor/Basketball')

## Create or connect to a local SQLite database

Place your CSVs in data/raw (e.g. player_stats.csv, salaries.csv). The code below will create data/database.db and load any CSVs it finds into tables named after the CSV (without extension).

In [ ]:
data_dir = repo_root / 'data'
raw_dir = data_dir / 'raw'
db_path = data_dir / 'database.db'
data_dir.mkdir(parents=True, exist_ok=True)
raw_dir.mkdir(parents=True, exist_ok=True)

engine = create_engine(f"sqlite:///{db_path}")

# Ingest CSVs found in data/raw into SQLite (table name = filename without .csv)
for csv_file in raw_dir.glob('*.csv'):
    table_name = csv_file.stem
    print(f'Loading {csv_file} -> table: {table_name}')
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, engine, if_exists='replace', index=False)

print('Database ready at:', db_path)

## Example: season-level aggregation with SQL

Adjust column/table names to match your CSV schema. This example assumes a table `player_stats` with columns: player_id, season, game_id, points, rebounds, assists.

In [ ]:
example_query = '''
SELECT player_id,
       season,
       COUNT(DISTINCT game_id) AS games,
       SUM(points) AS total_points,
       SUM(rebounds) AS total_rebounds,
       SUM(assists) AS total_assists
FROM player_stats
GROUP BY player_id, season
ORDER BY season DESC
LIMIT 100;
'''

try:
    season_df = pd.read_sql_query(example_query, engine)
    season_df['pts_per_game'] = season_df['total_points'] / season_df['games']
    season_df['reb_per_game'] = season_df['total_rebounds'] / season_df['games']
    season_df['ast_per_game'] = season_df['total_assists'] / season_df['games']
    season_df.head()
except Exception as e:
    print('Error running example query — check table/column names:', e)

## Next steps (Part 1 -> Part 2)
- Verify and standardize column names in your CSVs (player ids, season format, game ids).
- Write SQL joins to combine stats + salary tables and produce your target variable (e.g., season win shares, efficiency, or a salary gap).
- Save useful derived tables in the DB (e.g., season_aggregates, player_career_summaries) to speed iteration.
- For Part 2: design feature queries (rolling averages, age curves) using window functions or precomputed rolling tables.

If you share a sample of your CSV column names I can add tailored SQL queries next.